In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# Load ams model

In [2]:
ssp = ams.load('ams_case123_REGCV1_v6.xlsx')

ssp.LDOPF.setup()

ssp.LDOPF.om.mdl.solve()

5.112563139168072

In [3]:
ssp.LDOPF.obj.v

5.112563139168072

In [4]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [5]:
pg_ams = ssp.LDOPF.om.pg.value

pg_ams

array([0.04999101, 0.08999101, 0.07708993, 0.01999101, 0.02399101,
       0.01399101, 0.01199101, 0.01799101, 0.01499101, 0.01599101,
       0.01699101])

In [6]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9',
 'PV_10',
 'PV_11']

# Define new variables, constraints, and obj

In [7]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [8]:
constraints = constriants_ams

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [9]:
# Cost data

c2 = np.array([0.01, 0.01, 0.02, 0, 0, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
c1 = np.array([  20,   20,   30, 0, 0, 0, 0, 0, 0, 0, 0]) # gen linear cost

pg = ssp.LDOPF.om.pg

# pg_power = cp.power(pg, 2)
pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg)

obj = cp.Minimize(cost)

In [10]:
ssp.LDOPF.c2.v

array([0.01, 0.01, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [11]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

In [12]:
# built-in solver: ECOS_BB
# prob.solve(solver=cp.ECOS_BB, verbose=True)
# prob.solve(solver=cp.ECOS_BB)
prob.solve(solver=cp.GUROBI)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-13


5.110224580001787

In [13]:
ssp.LDOPF.c1

RParam: c1 <GCost>, v=[20. 20. 30.  0.  0.  0.  0.  0.  0.  0.  0.]

In [14]:
ssp.LDOPF.c2

RParam: c2 <GCost>, v=[0.01 0.01 0.02 0.   0.   0.   0.   0.   0.   0.   0.  ]

In [15]:
ssp.LDOPF.om.pg.value

array([0.05 , 0.09 , 0.077, 0.02 , 0.024, 0.014, 0.012, 0.018, 0.015,
       0.016, 0.017])

In [16]:
sum(ssp.LDOPF.om.pg.value)

0.35300000000000015

In [17]:
# check lower limit of pg
ssp.LDOPF.pmin.v

array([0.025, 0.025, 0.025, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   ])

In [18]:
# check upper limit of pg
ssp.LDOPF.pmax.v

array([0.05 , 0.09 , 0.2  , 0.02 , 0.024, 0.014, 0.012, 0.018, 0.015,
       0.016, 0.017])

In [19]:
c_M  = np.array([3, 4, 1, 1, 2, 1, 1, 1])

c_D  = np.array([2, 3, 1, 1, 1.5, 1, 1, 1])

In [20]:
M_I   = np.array([6, 6, 6, 6, 6, 6, 6, 6])
M_II  = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

D_I   = np.array([4, 4, 4, 4, 4, 4, 4, 4])
D_II  = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

M_III = np.array([0.11814667, 0.16732237, 0.68718068, 0.64605445, 0.47454199, 0.51813483, 0.61832465, 0.65060247])
D_III = np.array([0.12208462, 0.10709857, 0.16479399, 0.15698404, 0.17926447, 0.16375611, 0.1636579 , 0.16812373])


M_IV = np.array([1.318     , 1.317     , 1.737     , 1.696     , 1.474     ,  3.05397107, 5.43976424, 5.5       ])
D_IV = np.array([0.572, 0.157, 1.414, 1.406, 0.679, 1.413, 1.41 , 1.518])

In [25]:
Inertia_cost_I = sum(c_M * M_I)
Inertia_cost_II = sum(c_M * M_II)
Inertia_cost_III = sum(c_M * M_III)
Inertia_cost_IV = sum(c_M * M_IV)

Inertia_cost = [Inertia_cost_I, Inertia_cost_II, Inertia_cost_III, Inertia_cost_IV]
Inertia_cost

[84, 0.14, 5.0931105500000005, 29.59673531]

In [26]:
damping_cost_I = sum(c_D * D_I)
damping_cost_II = sum(c_D * D_II)
damping_cost_III = sum(c_D * D_III)
damping_cost_IV = sum(c_D * D_IV)

damping_cost = [damping_cost_I, damping_cost_II, damping_cost_III, damping_cost_IV]
damping_cost

[46.0, 0.11499999999999999, 1.651677425, 9.794500000000001]

In [ ]:
Total_cost = np.array(Inertia_cost) + np.array(damping_cost)

# 